# Fehlende Adressen durch Telefonbuch lassen mit API

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
import re
import bs4 as bs
import requests
import bs4 as bs
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
from geopy.geocoders import Nominatim #zum Geodaten hinzufügen
from geopy.exc import GeocoderTimedOut #zum excepteion umgehen
from ratelimit import limits, sleep_and_retry #zum Geodaten hinzufügen
from pandas import Panel
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"
import json

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version


In [32]:
urlanf="https://tel.search.ch/api/?wo="
adress="Zugerstrasse 76B, 6340 Baar"
urlschl="&key=YYYYYY"


In [33]:
response = requests.get(urlanf+adress+urlschl)

In [34]:
text= response.text

In [35]:
text

'<?xml version="1.0" encoding="utf-8" ?>\n<feed xml:lang="de" xmlns="http://www.w3.org/2005/Atom" xmlns:openSearch="http://a9.com/-/spec/opensearchrss/1.0/" xmlns:tel="http://tel.search.ch/api/spec/result/1.0/">\n  <id>https://tel.search.ch/api/3ef7e218e8a6d4c8084ee2d4ee0bd9a4/65a6ed91a725144c790dfbd8a109103a</id>\n  <title type="text">tel.search.ch API Search Results</title>\n  <generator version="1.0" uri="https://tel.search.ch">tel.search.ch</generator>\n  <updated>2020-03-01T03:00:00Z</updated>\n  <link href="https://tel.search.ch/result.html?wo=Zugerstrasse+76B,+6340+Baar" rel="alternate" type="text/html" />\n  <link href="http://tel.search.ch/api/?wo=Zugerstrasse+76B,+6340+Baar&amp;key=3ef7e218e8a6d4c8084ee2d4ee0bd9a4" type="application/atom+xml" rel="self" />\n  <link href="http://tel.search.ch/api/?wo=Zugerstrasse+76B,+6340+Baar&amp;pos=11&amp;key=3ef7e218e8a6d4c8084ee2d4ee0bd9a4" rel="next" type="application/atom+xml" />\n  <openSearch:totalResults>29</openSearch:totalResults>

In [26]:
df_fehlende_adressen_sauber=pd.read_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_geocode_einzeln.csv")

In [30]:
telfelist=[]
for adress in tqdm(df_fehlende_adressen_sauber["Adresse"]):
    response = requests.get(urlanf+adress+urlschl)
    text= response.text
   
    
    minidict_tel={"output tel":text, "Adresse PLZ":adress}
    telfelist.append(minidict_tel)
   
    


100%|██████████| 3713/3713 [12:20<00:00,  5.02it/s]


In [31]:
df_fehlendetel=pd.DataFrame(telfelist)
df_fehlendetel.to_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_geocode_und_telbucheintraegen.csv")
df_fehlendetel

,output tel,Adresse PLZ
0,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Haldenstrasse 5, 6340 Baar"
1,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Gewerbestrasse 5, 6330 Cham"
2,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Neuhofstrasse 5A, 6340 Baar"
3,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Zugerstrasse 76B, 6340 Baar"
4,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Alte Steinhauserstrasse 1, 6330 Cham"
...,...,...
3708,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobstrasse 81A, 6330 Cham"
3709,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobstrasse 85A, 6333 Hünenberg See"
3710,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobweid 2, 6333 Hünenberg See"
3711,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobweid 3, 6333 Hünenberg See"


In [39]:
# Nun die Anzahl extrahieren
df_fehlendetel["Anzahl"]=df_fehlendetel["output tel"].str.extract("\<openSearch\:totalResults\>(.*)\<\/openSearch\:totalResults")

In [41]:
df_fehlendetel # Hier nun die Anzahl Telefonbucheinträge

,output tel,Adresse PLZ,Anzahl
0,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Haldenstrasse 5, 6340 Baar",44
1,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Gewerbestrasse 5, 6330 Cham",51
2,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Neuhofstrasse 5A, 6340 Baar",34
3,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Zugerstrasse 76B, 6340 Baar",29
4,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Alte Steinhauserstrasse 1, 6330 Cham",82
...,...,...,...
3708,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobstrasse 81A, 6330 Cham",0
3709,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobstrasse 85A, 6333 Hünenberg See",1
3710,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobweid 2, 6333 Hünenberg See",1
3711,"<?xml version=""1.0"" encoding=""utf-8"" ?>\n<feed...","Huobweid 3, 6333 Hünenberg See",7


In [43]:
df_fehlendetel.to_csv(folderpath+"/BRFrames/Adressen_alle3713_mit_Anzahl_telbucheintraegen_extrahiert.csv")